# Audio – Spectrogram CNN

# 🔊 Audio – Simple Spectrogram Classification

## Dữ liệu: ../data/audio/class_a|class_b/*.wav

In [ ]:
!pip install -U tensorflow keras librosa soundfile

In [ ]:
import os, numpy as np, matplotlib.pyplot as plt, librosa, librosa.display, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from glob import glob

root_drive = '/content/drive/MyDrive/keras_project/data/audio'
data_root = root_drive if tf.io.gfile.exists(root_drive) else '../data/audio'

paths_a = sorted(glob(os.path.join(data_root, 'class_a', '*.wav')))
paths_b = sorted(glob(os.path.join(data_root, 'class_b', '*.wav')))

def wav_to_logmel(p, sr=16000, n_mels=64):
    y, sr = librosa.load(p, sr=sr)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    logS = librosa.power_to_db(S, ref=np.max)
    return logS

X, y = [], []
for p in paths_a:
    X.append(wav_to_logmel(p)); y.append(0)
for p in paths_b:
    X.append(wav_to_logmel(p)); y.append(1)
X = np.array(X)[..., None].astype('float32')
y = np.array(y)

idx = np.arange(len(X)); np.random.seed(42); np.random.shuffle(idx)
split = int(0.8*len(idx))
tr, va = idx[:split], idx[split:]
X_tr, y_tr = X[tr], y[tr]; X_va, y_va = X[va], y[va]

model = keras.Sequential([
    layers.Conv2D(16, 3, activation='relu', input_shape=X.shape[1:]),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(X_tr, y_tr, validation_data=(X_va, y_va), epochs=8, batch_size=8)

plt.figure(); plt.plot(hist.history['accuracy'], label='acc'); plt.plot(hist.history['val_accuracy'], label='val_acc'); plt.legend(); plt.title('Accuracy'); plt.xlabel('epoch'); plt.show()
